In [1]:
from ilqr import iLQR
import gym
import numpy as np
import pandas as pd
import time

from aprl.agents import MujocoFiniteDiffDynamicsPerformance
from aprl.envs.mujoco_costs import get_cost
from experiments.common import set_seeds, make_env, fit_ilqr, \
                               on_iteration, receding, multi_evaluate

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


Logging to /tmp/openai-2019-02-02-10-10-00-610410
Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']
Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']


In [2]:
seed = 42
set_seeds(seed)

env_name = 'InvertedPendulum-v2'
env, us_init = make_env(env_name, seed=seed)
dynamics = {'performance': MujocoFiniteDiffDynamicsPerformance(env)}
x0s = {k: dyn.get_state() for k, dyn in dynamics.items()}

costs = {k: get_cost(env_name) for k in dynamics.keys()}
ilqrs = {k: iLQR(dyn, costs[k], len(us_init))  
         for k, dyn in dynamics.items()}

In [3]:
xs, us = fit_ilqr(ilqrs, x0s, us_init)

dict_keys(['performance'])
*** Fitting performance ***
iteration -1 converged 8000.546301728309 [-1.00891306e+00 -1.57327613e+00 -2.39078527e-01  3.66173836e-05] [1.0628469]
iteration 0 accepted 4163.297646900738 [ 9.86032424e-01 -1.57327643e+00 -7.09389833e-02 -4.13490150e-07] [-0.0660465]
iteration 1 accepted 4085.4790703680055 [ 9.99589959e-01 -1.57327637e+00 -2.24617126e-03 -1.29597510e-08] [-0.00208583]
iteration 2 accepted 4047.6417957739595 [ 9.99888747e-01 -1.57327637e+00 -1.70797153e-05 -1.16005145e-10] [-1.7101862e-05]
iteration 3 accepted 3511.784776912087 [ 9.96944675e-01 -1.57327637e+00 -1.16681165e-03 -3.57811689e-10] [-1.1345905e-05]
iteration 4 accepted 3129.1146728283534 [ 9.99885290e-01 -1.57327637e+00  1.35484353e-06 -1.20823857e-11] [-2.688765e-07]
iteration 5 accepted 3109.4496996000526 [ 9.99920827e-01 -1.57327637e+00  1.41322777e-06 -1.17482269e-11] [-1.9737696e-07]
iteration 6 accepted 3056.062515262745 [ 9.99936986e-01 -1.57327637e+00  1.41047656e-06 -1.1782571

# Receding horizon

In [5]:
for k, ilqr in ilqrs.items():
    underlying = 'receding_' + k
    dynamics[underlying] = dynamics[k]
    x0s[underlying] = x0s[k]
    xs[underlying], us[underlying] = receding(env, ilqr, x0s[k], us_init)

iteration 0 r = 1.0, x = [[-0.00251713 -0.00031323  0.00841542 -0.00036109]
 [-0.0033249   0.00199222 -0.08901676  0.22941983]], u = [[-0.5292973]]
iteration 1 r = 1.0, x = [[-0.0033249   0.00199222 -0.08901676  0.22941983]
 [-0.00561983  0.00774675 -0.14043524  0.34575193]], u = [[-0.28389445]]
iteration 2 r = 1.0, x = [[-0.00561983  0.00774675 -0.14043524  0.34575193]
 [-0.0086619   0.01515624 -0.16380512  0.39558472]], u = [[-0.13228847]]
iteration 3 r = 1.0, x = [[-0.0086619   0.01515624 -0.16380512  0.39558472]
 [-0.01199674  0.02316965 -0.16975357  0.40650024]], u = [[-0.03629446]]
iteration 4 r = 1.0, x = [[-0.01199674  0.02316965 -0.16975357  0.40650024]
 [-0.01533513  0.03116269 -0.164181    0.39372604]], u = [[0.02878692]]
iteration 5 r = 1.0, x = [[-0.01533513  0.03116269 -0.164181    0.39372604]
 [-0.01826437  0.03823806 -0.12888825  0.31514031]], u = [[0.19349981]]
iteration 6 r = 1.0, x = [[-0.01826437  0.03823806 -0.12888825  0.31514031]
 [-0.02056362  0.04397665 -0.1011

iteration 55 r = 1.0, x = [[ 0.13349496 -0.03650474 -0.00881304  0.06061798]
 [ 0.1332397  -0.03528492 -0.01672337  0.06147852]], u = [[-0.05613283]]
iteration 56 r = 1.0, x = [[ 0.1332397  -0.03528492 -0.01672337  0.06147852]
 [ 0.13282981 -0.03405095 -0.02427542  0.06203663]], u = [[-0.05386677]]
iteration 57 r = 1.0, x = [[ 0.13282981 -0.03405095 -0.02427542  0.06203663]
 [ 0.13227242 -0.03280859 -0.03147485  0.0623204 ]], u = [[-0.05161717]]
iteration 58 r = 1.0, x = [[ 0.13227242 -0.03280859 -0.03147485  0.0623204 ]
 [ 0.1315745  -0.03156307 -0.038328    0.06235602]], u = [[-0.04939021]]
iteration 59 r = 1.0, x = [[ 0.1315745  -0.03156307 -0.038328    0.06235602]
 [ 0.13074292 -0.03031908 -0.04484177  0.06216791]], u = [[-0.04719136]]
iteration 60 r = 1.0, x = [[ 0.13074292 -0.03031908 -0.04484177  0.06216791]
 [ 0.12978438 -0.02908088 -0.05102357  0.06177884]], u = [[-0.04502549]]
iteration 61 r = 1.0, x = [[ 0.12978438 -0.02908088 -0.05102357  0.06177884]
 [ 0.12870545 -0.027852

iteration 110 r = 1.0, x = [[ 0.02102157  0.00137681 -0.11897677  0.00607517]
 [ 0.01864961  0.00149277 -0.11822134  0.00553652]], u = [[0.00382741]]
iteration 111 r = 1.0, x = [[ 0.01864961  0.00149277 -0.11822134  0.00553652]
 [ 0.01629293  0.00159823 -0.11744843  0.00502502]], u = [[0.00398013]]
iteration 112 r = 1.0, x = [[ 0.01629293  0.00159823 -0.11744843  0.00502502]
 [ 0.01395186  0.00169374 -0.11666001  0.0045397 ]], u = [[0.00411825]]
iteration 113 r = 1.0, x = [[ 0.01395186  0.00169374 -0.11666001  0.0045397 ]
 [ 0.0116267   0.00177981 -0.11585821  0.00408027]], u = [[0.00424103]]
iteration 114 r = 1.0, x = [[ 0.0116267   0.00177981 -0.11585821  0.00408027]
 [ 0.00931768  0.00185695 -0.11504465  0.0036455 ]], u = [[0.00435137]]
iteration 115 r = 1.0, x = [[ 0.00931768  0.00185695 -0.11504465  0.0036455 ]
 [ 0.00702504  0.00192564 -0.11422104  0.00323458]], u = [[0.00444905]]
iteration 116 r = 1.0, x = [[ 0.00702504  0.00192564 -0.11422104  0.00323458]
 [ 0.00474895  0.00198

iteration 165 r = 1.0, x = [[-0.08573105  0.00110366 -0.07772642 -0.00176619]
 [-0.08728003  0.00106857 -0.07717083 -0.00174617]], u = [[0.00321831]]
iteration 166 r = 1.0, x = [[-0.08728003  0.00106857 -0.07717083 -0.00174617]
 [-0.08881796  0.00103389 -0.07662267 -0.00172539]], u = [[0.00317112]]
iteration 167 r = 1.0, x = [[-0.08881796  0.00103389 -0.07662267 -0.00172539]
 [-0.09034501  0.00099963 -0.07608186 -0.00170392]], u = [[0.00312451]]
iteration 168 r = 1.0, x = [[-0.09034501  0.00099963 -0.07608186 -0.00170392]
 [-0.09186131  0.0009658  -0.07554824 -0.00168192]], u = [[0.00307875]]
iteration 169 r = 1.0, x = [[-0.09186131  0.0009658  -0.07554824 -0.00168192]
 [-0.09336702  0.00093242 -0.07502173 -0.00165943]], u = [[0.00303356]]
iteration 170 r = 1.0, x = [[-0.09336702  0.00093242 -0.07502173 -0.00165943]
 [-0.09486226  0.00089949 -0.0745022  -0.00163653]], u = [[0.00298906]]
iteration 171 r = 1.0, x = [[-0.09486226  0.00089949 -0.0745022  -0.00163653]
 [-0.09634718  0.00086

iteration 217 r = 1.0, x = [[-1.54329515e-01 -8.63222167e-05 -5.63117520e-02 -6.19088433e-04]
 [-1.55452579e-01 -9.86904065e-05 -5.59946100e-02 -6.18778905e-04]], u = [[0.00170946]]
iteration 218 r = 1.0, x = [[-1.55452579e-01 -9.86904065e-05 -5.59946100e-02 -6.18778905e-04]
 [-1.56569374e-01 -1.10934371e-04 -5.56848842e-02 -6.06742811e-04]], u = [[0.0016681]]
iteration 219 r = 1.0, x = [[-1.56569374e-01 -1.10934371e-04 -5.56848842e-02 -6.06742811e-04]
 [-1.57679945e-01 -1.23068365e-04 -5.53721869e-02 -6.07675109e-04]], u = [[0.00168297]]
iteration 220 r = 1.0, x = [[-1.57679945e-01 -1.23068365e-04 -5.53721869e-02 -6.07675109e-04]
 [-1.58784337e-01 -1.35100804e-04 -5.50669359e-02 -5.96668066e-04]], u = [[0.00164151]]
iteration 221 r = 1.0, x = [[-1.58784337e-01 -1.35100804e-04 -5.50669359e-02 -5.96668066e-04]
 [-1.59882595e-01 -1.47039472e-04 -5.47588440e-02 -5.98196138e-04]], u = [[0.00165571]]
iteration 222 r = 1.0, x = [[-1.59882595e-01 -1.47039472e-04 -5.47588440e-02 -5.98196138e-0

...

iteration 815 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  1.99300231e-14 -1.37892541e-16]
 [-2.14470784e-01 -1.66666512e-03  1.88937130e-14  1.10646913e-15]], u = [[-6.31220841e-15]]
iteration 816 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  1.88937130e-14  1.10646913e-15]
 [-2.14470784e-01 -1.66666512e-03  1.69693569e-14  4.41779164e-15]], u = [[-1.11677102e-14]]
iteration 817 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  1.69693569e-14  4.41779164e-15]
 [-2.14470784e-01 -1.66666512e-03  1.74138894e-14  2.08906874e-15]], u = [[1.64481646e-15]]
iteration 818 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  1.74138894e-14  2.08906874e-15]
 [-2.14470784e-01 -1.66666512e-03  1.73628683e-14  1.03382947e-15]], u = [[-9.66299535e-16]]
iteration 819 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  1.73628683e-14  1.03382947e-15]
 [-2.14470784e-01 -1.66666512e-03  1.68701133e-14  1.04960570e-15]], u = [[-3.34541853e-15]]
iteration 820 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  1.68

iteration 859 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  2.29038753e-15  3.49297763e-15]
 [-2.14470784e-01 -1.66666512e-03  8.06358287e-16  6.39022240e-15]], u = [[-8.48282392e-15]]
iteration 860 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  8.06358287e-16  6.39022240e-15]
 [-2.14470784e-01 -1.66666512e-03  1.31505301e-15  4.56542692e-15]], u = [[2.31295659e-15]]
iteration 861 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  1.31505301e-15  4.56542692e-15]
 [-2.14470784e-01 -1.66666512e-03  2.30285154e-15  1.71628214e-15]], u = [[4.99866782e-15]]
iteration 862 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  2.30285154e-15  1.71628214e-15]
 [-2.14470784e-01 -1.66666512e-03  1.74349938e-15  2.61269784e-15]], u = [[-3.32652946e-15]]
iteration 863 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  1.74349938e-15  2.61269784e-15]
 [-2.14470784e-01 -1.66666512e-03  1.16710100e-15  3.54163817e-15]], u = [[-3.43038234e-15]]
iteration 864 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  1.167

iteration 903 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03 -9.02988116e-17  1.92120197e-15]
 [-2.14470784e-01 -1.66666512e-03 -1.21261470e-15  4.41717363e-15]], u = [[-6.20958075e-15]]
iteration 904 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03 -1.21261470e-15  4.41717363e-15]
 [-2.14470784e-01 -1.66666512e-03 -2.19047205e-16  1.90022044e-15]], u = [[5.25865646e-15]]
iteration 905 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03 -2.19047205e-16  1.90022044e-15]
 [-2.14470784e-01 -1.66666512e-03 -1.27365249e-15  4.30890532e-15]], u = [[-5.78981125e-15]]
iteration 906 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03 -1.27365249e-15  4.30890532e-15]
 [-2.14470784e-01 -1.66666512e-03  1.01175162e-15 -1.19542569e-15]], u = [[1.23205169e-14]]
iteration 907 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  1.01175162e-15 -1.19542569e-15]
 [-2.14470784e-01 -1.66666512e-03 -7.00949570e-16  2.87179040e-15]], u = [[-9.27494752e-15]]
iteration 908 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03 -7.009

iteration 947 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  9.82416963e-16  1.42852617e-15]
 [-2.14470784e-01 -1.66666512e-03  1.54451110e-15  1.48195621e-16]], u = [[3.09712256e-15]]
iteration 948 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  1.54451110e-15  1.48195621e-16]
 [-2.14470784e-01 -1.66666512e-03 -2.92018764e-16  4.56267013e-15]], u = [[-9.90186922e-15]]
iteration 949 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03 -2.92018764e-16  4.56267013e-15]
 [-2.14470784e-01 -1.66666512e-03  3.99827631e-16  2.92731633e-15]], u = [[3.75337348e-15]]
iteration 950 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  3.99827631e-16  2.92731633e-15]
 [-2.14470784e-01 -1.66666512e-03  2.18846883e-15 -1.19584662e-15]], u = [[9.79210276e-15]]
iteration 951 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  2.18846883e-15 -1.19584662e-15]
 [-2.14470784e-01 -1.66666512e-03  2.29796744e-15 -1.24422935e-15]], u = [[7.70923844e-16]]
iteration 952 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  2.29796

iteration 991 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  8.86012151e-16 -9.45063793e-16]
 [-2.14470784e-01 -1.66666512e-03  7.32508666e-16 -7.24887174e-16]], u = [[-9.29328496e-16]]
iteration 992 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  7.32508666e-16 -7.24887174e-16]
 [-2.14470784e-01 -1.66666512e-03  1.03305053e-15 -1.57793156e-15]], u = [[1.5343378e-15]]
iteration 993 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  1.03305053e-15 -1.57793156e-15]
 [-2.14470784e-01 -1.66666512e-03  8.43840045e-16 -1.26092856e-15]], u = [[-1.11260661e-15]]
iteration 994 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  8.43840045e-16 -1.26092856e-15]
 [-2.14470784e-01 -1.66666512e-03  6.24298408e-16 -9.18206352e-16]], u = [[-1.31293328e-15]]
iteration 995 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03  6.24298408e-16 -9.18206352e-16]
 [-2.14470784e-01 -1.66666512e-03 -1.04165065e-15  2.80998095e-15]], u = [[-9.19279472e-15]]
iteration 996 r = 1.0, x = [[-2.14470784e-01 -1.66666512e-03 -1.041

# Rollouts

In [8]:
multi_evaluate(env, dynamics, x0s, us)

performance
receding_performance


,rewards,lengths
performance,90.0,90
receding_performance,232.0,232
